# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.


## Section 1: Create a dataframe of Toronto postal code

1. Start by creating a new Notebook for this assignment.
2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 
3. To create the above dataframe:


* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.


In [1]:
#!pip install beautifulsoup4 # Install package for web scrapping
#!pip install lxml #Install parcer? for 
#!pip install html5lib

from bs4 import BeautifulSoup
import requests
import pandas as pd
source = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050.').text
soup = BeautifulSoup(source,'lxml')
obj=soup.find('tr')
#print(obj.prettify())
data=pd.DataFrame(columns=['Postal Code', 'Borough','Neighborhood']) #create blank data frame for scrapping
list=[]
for obj in soup.find_all('tr'):
    list=[]
    try:
        for tr in obj.find_all('td'):
            #print(tr.text)
            list=list+[tr.text]
    except Exception as e:
        list=[]
    if len(list)==3:
        #print(list)
        data=data.append({'Postal Code': list[0], 'Borough': list[1],'Neighborhood': list[2]},ignore_index=True)
        #print(data)
#data.head()
print('Total number of M-postal code:',data.shape[0])
df=data[data.Borough != 'Not assigned']
df['Neighborhood']= df['Neighborhood'].str.replace('\n', '')
print('Total number of assigned M-postal code:',df.shape[0])
df.head()

Total number of M-postal code: 287
Total number of assigned M-postal code: 210


/Users/lbpham/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [2]:
df.shape

(210, 3)

## Section 2: Adding geo-coordinates to the dataframe

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Important Note: There is a limit on how many times you can call geocoder.google function. It is 2500 times per day. This should be way more than enough for you to get acquainted with the package and to use it to get the geographical coordinates of the neighborhoods in the Toronto.

Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

In [3]:
#!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None
# loop until you get the coordinates
#while(lat_lng_coords is None):
#g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
#lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print(lat_lng_coords)

url='https://cocl.us/Geospatial_data'
geo = pd.read_csv(url)
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
final=pd.merge(df, geo, on='Postal Code')
final=final.sort_values(by=['Postal Code']).reset_index(drop=True)
final.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Port Union,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Highland Creek,43.784535,-79.160497


## Section 3: Explore and cluster the neighborhoods in Toronto


In [6]:
#Import all the libraries needed
#!pip install geopy
#!pip install folium

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [7]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [8]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for i in range(final.shape[0]):
    neighborhood=final.loc[i,'Neighborhood']
    borough=final.loc[i,'Borough']
    lat=final.loc[i,'Latitude']
    lng=final.loc[i,'Longitude']
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [10]:
toronto=final[final['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,Riverdale,43.679557,-79.352188
2,M4K,East Toronto,The Danforth West,43.679557,-79.352188
3,M4L,East Toronto,The Beaches West,43.668999,-79.315572
4,M4L,East Toronto,India Bazaar,43.668999,-79.315572


In [182]:
import matplotlib.pyplot as plt
import random

area=toronto.Borough.unique()
color={}

for i in range(len(area)):
    #print(area[i])
    color[area[i]]="#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    #print(color[area[i]])
color                                

{'East Toronto': '#E1B478',
 'Central Toronto': '#23F3D0',
 'Downtown Toronto': '#996B9E',
 'West Toronto': '#0D4B98'}

In [188]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=12)
# add markers to map
for i in range(toronto.shape[0]):
    label='N: ' + toronto.loc[i,'Neighborhood'] + ' | ' + 'B: '+toronto.loc[i,'Borough']
    lat=toronto.loc[i,'Latitude']
    lng=toronto.loc[i,'Longitude']
    cluster=toronto.loc[i,'Borough']
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color[toronto.loc[i,'Borough']],
        fill=True,
        fill_color=color[toronto.loc[i,'Borough']],
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  

### Map of Toronto with Neighborhood clustered by Borough

In [190]:
map_downtown

**Define Foursquare Credentials and Version**

In [12]:
CLIENT_ID = '1FVLOJLQSDTHT41K3XVQUTWC0ONHKLCT0KT1FRO0KSKYXR01' # your Foursquare ID
CLIENT_SECRET = '532GTN1I43GCT1N2H3GCUHQQPOA2SYO04JVYQEINRH5J1VGX' # your Foursquare Secret
VERSION = '20190320'

toronto.loc[0, 'Neighborhood']

'The Beaches'

In [13]:
neighborhood_latitude = toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [14]:
url='https://api.foursquare.com/v2/venues/search?ll={},{}&client_id={}&client_secret={}&v={}'.format(neighborhood_latitude,
                                                                                                     neighborhood_longitude,
                                                                                                     CLIENT_ID,
                                                                                                     CLIENT_SECRET,
                                                                                                     VERSION)
url

'https://api.foursquare.com/v2/venues/search?ll=43.67635739999999,-79.2930312&client_id=1FVLOJLQSDTHT41K3XVQUTWC0ONHKLCT0KT1FRO0KSKYXR01&client_secret=532GTN1I43GCT1N2H3GCUHQQPOA2SYO04JVYQEINRH5J1VGX&v=20190320'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e758e886d8c56001bb4e001'},
 'response': {'venues': [{'id': '56afcad6498e05333bf42031',
    'name': 'Glen Stewart Ravine',
    'location': {'lat': 43.67629984029563,
     'lng': -79.2947841389563,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.67629984029563,
       'lng': -79.2947841389563}],
     'distance': 141,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d162941735',
      'name': 'Other Great Outdoors',
      'pluralName': 'Other Great Outdoors',
      'shortName': 'Other Outdoors',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/outdoors_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1584762561',
    'hasPerk': False},
   {'id': '4bd461bc77b29c74a07d9282',
    'name': 'Glen Manor Ravine',
    'location': {'address': 'Glen Manor',
     'crossStreet': 'Queen St.',
     'lat': 43.67682094413784,


In [33]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [141]:
venues = results['response']['venues']    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.columns=('Name','Cat','Lat','Lng','Categories')
nearby_venues=nearby_venues.drop(columns='Cat')

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

30 venues were returned by Foursquare.


,Name,Lat,Lng,Categories
0,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
1,Glen Manor Ravine,43.676821,-79.293942,Trail
2,Kew-Balmy Beach,43.667372,-79.295312,Beach
3,Glen Stewart Park,43.675278,-79.294647,Park
4,Intuitive Touch,43.679729,-79.288732,None


### Explore Neighborhoods in Manhattan

In [86]:
toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,Riverdale,43.679557,-79.352188
2,M4K,East Toronto,The Danforth West,43.679557,-79.352188
3,M4L,East Toronto,The Beaches West,43.668999,-79.315572
4,M4L,East Toronto,India Bazaar,43.668999,-79.315572


In [144]:
CLIENT_ID = '1FVLOJLQSDTHT41K3XVQUTWC0ONHKLCT0KT1FRO0KSKYXR01' # your Foursquare ID
CLIENT_SECRET = '532GTN1I43GCT1N2H3GCUHQQPOA2SYO04JVYQEINRH5J1VGX' # your Foursquare Secret
VERSION = '20190320'

def getNearbyVenues(code, names, lat, lng):
    venues_list=pd.DataFrame()
    for i in range(len(names)):
        #print(code[i])
        #print(names[i])
        #print(lat[i])
        #print(lng[i])
            
        # create the API request URL
        url='https://api.foursquare.com/v2/venues/search?ll={},{}&client_id={}&client_secret={}&v={}'.format(lat[i],
                                                                                                             lng[i],
                                                                                                             CLIENT_ID,
                                                                                                             CLIENT_SECRET,
                                                                                                             VERSION)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        results=json_normalize(results)
        #print(results)
        
        # return only relevant information for each nearby venue
        filtered_columns = ['id','name', 'categories', 'location.lat', 'location.lng']
        nearby_venues =results.loc[:, filtered_columns]
        #print(nearby_venues)
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
        nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        nearby_venues.columns=('id','Name','Cat','Lat','Lng','Categories')
        nearby_venues=nearby_venues.drop(columns='Cat')
        nearby_venues['Postal Code']=code[i]
        nearby_venues['Neighborhood']=names[i]
        #print(nearby_venues)
        venues_list=venues_list.append(nearby_venues,ignore_index=True)
    return(venues_list)
        
        
#        venues_list.append([(
#            code[i],
#            names[i], 
 #           lat[i], 
  #          lng[i], 
   #         v['venue']['name'], 
    #        v['venue']['location']['lat'], 
     #       v['venue']['location']['lng'],  
      #      v['venue']['categories'][0]['name']) for v in results])

#    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#    nearby_venues.columns = ['Neighborhood', 
#                  'Neighborhood Latitude', 
#                  'Neighborhood Longitude', 
#                  'Venue', 
#                  'Venue Latitude', 
#                  'Venue Longitude', 
#                  'Venue Category']
    
#    return(nearby_venues)

In [148]:
toronto_venues = getNearbyVenues(code=toronto['Postal Code'],
                                 names=toronto['Neighborhood'],
                                 lat=toronto['Latitude'],
                                 lng=toronto['Longitude'])
print('original: ',toronto_venues.shape)
toronto_venues=toronto_venues.drop_duplicates()
print('after remove duplicate: ',toronto_venues.shape)
toronto_venues.head()

original:  (2220, 7)
after remove duplicate:  (2220, 7)


,id,Name,Lat,Lng,Categories,Postal Code,Neighborhood
0,56afcad6498e05333bf42031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors,M4E,The Beaches
1,4bd461bc77b29c74a07d9282,Glen Manor Ravine,43.676821,-79.293942,Trail,M4E,The Beaches
2,4bf9a4ac4a67c928773a27cf,Kew-Balmy Beach,43.667372,-79.295312,Beach,M4E,The Beaches
3,4b1c19f9f964a520240224e3,Intuitive Touch,43.679729,-79.288732,None,M4E,The Beaches
4,4dbc8fe96a23e294ba3237bd,Glen Stewart Park,43.675278,-79.294647,Park,M4E,The Beaches


## DRAFT

The following script is for scrapping the Postal code from the link provided in the lab instruction.
The result from this script is different from the one above, less code retrieved so I decided to use the one above instead of this one

In [ ]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
obj=soup.find('td')
#print(obj.prettify())
code=obj.p.b.text
code

In [ ]:
# Scrapping data from the website
data=pd.DataFrame(columns=['Postal Code', 'Borough','Neighborhood']) #create blank data frame for scrapping
for obj in soup.find_all('td'):
    try:
        code=obj.p.b.text
        #print(code)
        #Find borough
        try:
            for href in obj.find_all('span'):
                br=href.a.text
                #print(href.a.text)
        except Exception as e:
            br=""
            #print('')
        
        #Find neighborhood    
        try:
            nb=obj.span.text
            #print(nb)
        except Exception as e:
            nb=obj.span.a.text
            #print(nb)
        #Write the data into dataframe
        data=data.append({'Postal Code': code, 'Borough': br,'Neighborhood': nb},ignore_index=True)
    except Exception as e:
        code=""
        boro=""
print('Total number of M-postal code:',data.shape[0])
data.tail(10)

**Remove the postal codes that are not assigned**

In [ ]:
df=data[data.Neighborhood != 'Not assigned']
df=df.reset_index(drop=True)
print('Total number of assigned M-postal code:',df.shape[0])
df.head()

In [ ]:
#Split the neighbor column into borough and neighborhood information
new=df["Neighborhood"].str.split("(", n = 1, expand = True)
new.columns=['Borough1','Neighborhood']

In [ ]:
#Select the data after splitting
borough=new["Borough1"].str.replace('/', ',')
neighbor=new["Neighborhood"].str.split(")", n = 1, expand = True)
neighbor.columns=['Neighborhood','other']
neighbor= neighbor['Neighborhood'].str.replace('/', ',')
neighbor.head()

In [ ]:
# Process the data to create final data frame
df.drop(columns='Neighborhood')
df["Borough1"]=borough
df["Neighborhood"]=neighbor
#df.tail()East Toronto
df.loc[100,'Borough']='East Toronto'
tem=df
length=df.shape[0]
for i in range(length):
    if tem.loc[i,'Borough'] not in tem.loc[i,'Borough1']:
        tem.loc[i,'Borough']=tem.loc[i,'Borough1']
    if tem.loc[i,'Neighborhood']==None:
        tem.loc[i,'Neighborhood']=tem.loc[i,'Borough1']
tem=tem.drop(columns='Borough1')
print('The number of rows of dataframe',tem.shape[0])
tem.head(10)